In [16]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="gemini-flash-latest",
    temperature=0
)


In [17]:
import pytest
from email_dataset import email_inputs, expected_tool_calls, triage_outputs_list, response_criteria_list

test_case_ix = 0

print("Email Input:", email_inputs[test_case_ix])
print("Expected Triage Output:", triage_outputs_list[test_case_ix])
print("Expected Tool Calls:", expected_tool_calls[test_case_ix])
print("Response Criteria:", response_criteria_list[test_case_ix])

from utils import format_messages_string
from email_assistant import email_assistant
from utils import extract_tool_calls

from langsmith import testing as t

@pytest.mark.langsmith
@pytest.mark.parametrize(
    "email_input, expected_calls",
    [   # Pick some examples with e-mail reply expected
        (email_inputs[0],expected_tool_calls[0]),
        (email_inputs[3],expected_tool_calls[3]),
    ],
)
def test_email_dataset_tool_calls(email_input, expected_calls):
    """Test if email processing contains expected tool calls.
    
    This test confirms that all expected tools are called during email processing,
    but does not check the order of tool invocations or the number of invocations
    per tool. Additional checks for these aspects could be added if desired.
    """
    # Run the email assistant
    messages = [{"role": "user", "content": str(email_input)}]
    result = email_assistant.invoke({"messages": messages})
            
    # Extract tool calls from messages list
    extracted_tool_calls = extract_tool_calls(result['messages'])
            
    # Check if all expected tool calls are in the extracted ones
    missing_calls = [call for call in expected_calls if call.lower() not in extracted_tool_calls]
    
    t.log_outputs({
                "missing_calls": missing_calls,
                "extracted_tool_calls": extracted_tool_calls,
                "response": format_messages_string(result['messages'])
            })

    # Test passes if no expected calls are missing
    assert len(missing_calls) == 0

Email Input: {'author': 'Alice Smith <alice.smith@company.com>', 'to': 'Lance Martin <lance@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': "Hi Lance,\n\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\n\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\n\nThanks!\nAlice"}
Expected Triage Output: respond
Expected Tool Calls: ['write_email', 'done']
Response Criteria: 
• Send email with write_email tool call to acknowledge the question and confirm it will be investigated  



In [18]:
from langsmith import Client

from email_dataset import examples_triage

# Initialize LangSmith client
client = Client()

# Dataset name
dataset_name = "E-mail Triage Evaluation"

# Create dataset if it doesn't exist
if not client.has_dataset(dataset_name=dataset_name):
    dataset = client.create_dataset(
        dataset_name=dataset_name, 
        description="A dataset of e-mails and their triage decisions."
    )
    # Add examples to the dataset
    client.create_examples(dataset_id=dataset.id, examples=examples_triage)

In [19]:
def target_email_assistant(inputs: dict) -> dict:
    """Process an email through the workflow-based email assistant."""
    response = email_assistant.nodes['triage_router'].invoke({"email_input": inputs["email_input"]})
    return {"classification_decision": response.update['classification_decision']}

In [20]:
def classification_evaluator(outputs: dict, reference_outputs: dict) -> bool:
    """Check if the answer exactly matches the expected answer."""
    return outputs["classification_decision"].lower() == reference_outputs["classification"].lower()

In [21]:
run_expt = True
if run_expt:
    experiment_results_workflow = client.evaluate(
        # Run agent 
        target_email_assistant,
        # Dataset name   
        data=dataset_name,
        # Evaluator
        evaluators=[classification_evaluator],
        # Name of the experiment
        experiment_prefix="E-mail assistant workflow", 
        # Number of concurrent evaluations
        max_concurrency=2, 
    )

View the evaluation results for experiment: 'E-mail assistant workflow-fc4c7001' at:
https://smith.langchain.com/o/e819c19c-8fe1-4341-b37f-842fdba883d5/datasets/26710b80-4a33-4f9b-8056-4afcacbe450e/compare?selectedSessions=dd18fbba-47fb-4f4e-9287-3664c9046331


🔔 Classification: NOTIFY - This email contains important information
🚫 Classification: IGNORE - This email can be safely ignored
📧 Classification: RESPOND - This email requires a response
🚫 Classification: IGNORE - This email can be safely ignored
🔔 Classification: NOTIFY - This email contains important information
🚫 Classification: IGNORE - This email can be safely ignored


Error running target function: Error calling model 'gemini-flash-latest' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 5, model: gemini-2.5-flash\nPlease retry in 428.998527ms.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel

🚫 Classification: IGNORE - This email can be safely ignored
📧 Classification: RESPOND - This email requires a response
🔔 Classification: NOTIFY - This email contains important information
📧 Classification: RESPOND - This email requires a response
📧 Classification: RESPOND - This email requires a response
📧 Classification: RESPOND - This email requires a response


In [ ]:
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI

class CriteriaGrade(BaseModel):
    """Score the response against specific criteria."""
    justification: str = Field(description="The justification for the grade and score, including specific examples from the response.")
    grade: bool = Field(description="Does the response meet the provided criteria?")
    
# Create a global LLM for evaluation to avoid recreating it for each test
criteria_eval_llm = ChatGoogleGenerativeAI(
    model="gemini-flash-latest",
    temperature=0
)
criteria_eval_structured_llm = criteria_eval_llm.with_structured_output(CriteriaGrade)

In [ ]:
email_input = email_inputs[0]
print("Email Input:", email_input)
success_criteria = response_criteria_list[0]
print("Success Criteria:", success_criteria)

In [ ]:
response = email_assistant.invoke({"email_input": email_input})

In [ ]:
from prompts import RESPONSE_CRITERIA_SYSTEM_PROMPT

all_messages_str = format_messages_string(response['messages'])
eval_result = criteria_eval_structured_llm.invoke([
        {"role": "system",
            "content": RESPONSE_CRITERIA_SYSTEM_PROMPT},
        {"role": "user",
            "content": f"""\n\n Response criteria: {success_criteria} \n\n Assistant's response: \n\n {all_messages_str} \n\n Evaluate whether the assistant's response meets the criteria and provide justification for your evaluation."""}
    ])

eval_result